## O1

In [28]:
import sklearn as sk
import numpy as np
import pandas as pd
import seaborn as sns
# from ydata_profiling import ProfileReport
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score #, confusion_matrix
from sklearn.metrics import r2_score #, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score, classification_report, confusion_matrix, matthews_corrcoef
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
import scipy
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [29]:
def convert_to_target(diagnoses):
    if diagnoses == '-':
        return 'healthy'
    elif diagnoses in 'ABCD':
        return 'hyperthyroid condition'
    elif diagnoses in 'EFGH':
        return 'hypothyroid condition'
    elif diagnoses in 'IJ':
        return 'binding protein'
    elif diagnoses in 'K':
        return 'general health'
    elif diagnoses in 'LMN':
        return 'replacement therapy'
    elif diagnoses == 'R':
        return 'discordant results'
    else:
        return 'other'

In [30]:
import warnings
warnings.filterwarnings("ignore")
data = pd.read_csv("../proj-data-c.csv")
data = data.drop(columns=['referral source:'])
data = data.drop(columns=['[record identification]'])
data = data[data['sex:'] != '?']
columns_with_question_mark = data.columns[data.apply(lambda col: (col == '?').any())]
for column in columns_with_question_mark:
    data[column] = data[column].replace('?', np.nan)
data['target'] = data['diagnoses'].apply(convert_to_target)
data = data[data['age:'] <= 100]
y = data['target']
X = data.drop(columns=['diagnoses'])
X = X.drop(columns=['target'])
np.random.seed(0)
major_class = y.value_counts().idxmax()
major_class_indices = y[y == major_class].index
drop_indices = np.random.choice(major_class_indices, size=int(len(major_class_indices)/2), replace=False)
X = X.drop(drop_indices)
y = y.drop(drop_indices)
X = X.replace('f', 0)
X = X.replace('t', 1)
X = X.replace('M', 0)
X = X.replace('F', 1)

X = X.fillna(-1)

In [31]:
import warnings
warnings.filterwarnings("ignore")
le = LabelEncoder()
y = le.fit_transform(y)
models = {'LogisticRegression': LogisticRegression(),'XGBoost': XGBClassifier(),'RandomForest': RandomForestClassifier(n_estimators=100),
    'DecisionTree': DecisionTreeClassifier(),'SVM': SVC(),'KNN': KNeighborsClassifier(n_neighbors=10, weights='distance')}
selected_features = {}
for name, model in models.items():
    start_time = time.time()
    sfs = SequentialFeatureSelector(model, n_features_to_select=8, n_jobs=-1)
    sfs.fit(X, y)
    mask = sfs.get_support()
    selected_features[name] = X.columns[mask]
    end_time = time.time()
    elapsed_time = end_time - start_time
    #print(f'Features for {name}: {selected_features[name]}')
    #print(f'Time taken for {name}: {elapsed_time} seconds\n')


In [32]:
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
selected_features={'LogisticRegression': ['on thyroxine:', 'query hyperthyroid:', 'TSH:', 'T3 measured:', 'T3:','T4U measured:', 'T4U:', 'TBG:'],
                   'XGBoost':['on thyroxine:', 'thyroid surgery:', 'TSH:', 'T3:', 'TT4:', 'T4U:','FTI:', 'TBG:'],
                   'RandomForest': ['on thyroxine:', 'pregnant:', 'TSH:', 'T3:', 'TT4:', 'T4U:', 'FTI:','TBG:'],
                   'DecisionTree':['sex:', 'on thyroxine:', 'thyroid surgery:', 'TSH:', 'T3:', 'T4U:','FTI:', 'TBG:'],
                   'SVM': ['sex:', 'on thyroxine:', 'query on thyroxine:','on antithyroid medication:', 'sick:', 'TSH:', 'T3:', 'TBG:'],
                   'KNN': ['on thyroxine:', 'on antithyroid medication:', 'pregnant:', 'tumor:','TSH:', 'T3:', 'T4U:', 'TBG:']}
models = {'LogisticRegression': LogisticRegression(),'XGBoost': XGBClassifier(),'RandomForest': RandomForestClassifier(n_estimators=100),
    'DecisionTree': DecisionTreeClassifier(),'SVM': SVC(),'KNN': KNeighborsClassifier(n_neighbors=10, weights='distance'),}
imputers = [('SimpleImputer', SimpleImputer(strategy='mean')), ('KNNImputer', KNNImputer(n_neighbors=2, weights="uniform"))]
scalers = [('StandardScaler', StandardScaler()), ('MinMaxScaler', MinMaxScaler())]
for model_name, model in models.items():
    best_features = selected_features[model_name]
    X_best = X[best_features]
    X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.2, random_state=42)
    for imputer_name, imputer in imputers:
        X_train_imputed = imputer.fit_transform(X_train)
        X_test_imputed = imputer.transform(X_test)
        for scaler_name, scaler in scalers:
            X_train_scaled = scaler.fit_transform(X_train_imputed)
            X_test_scaled = scaler.transform(X_test_imputed)
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            print(f'Average of the f1 score {model_name} with {imputer_name} and {scaler_name}: {f1_score(y_test, y_pred, average='macro')}')    

Average of the f1 score LogisticRegression with SimpleImputer and StandardScaler: 0.5451400373038456
Average of the f1 score LogisticRegression with SimpleImputer and MinMaxScaler: 0.23449081889254786
Average of the f1 score LogisticRegression with KNNImputer and StandardScaler: 0.5451400373038456
Average of the f1 score LogisticRegression with KNNImputer and MinMaxScaler: 0.23449081889254786
Average of the f1 score XGBoost with SimpleImputer and StandardScaler: 0.8468581602743479
Average of the f1 score XGBoost with SimpleImputer and MinMaxScaler: 0.8468581602743479
Average of the f1 score XGBoost with KNNImputer and StandardScaler: 0.8468581602743479
Average of the f1 score XGBoost with KNNImputer and MinMaxScaler: 0.8468581602743479
Average of the f1 score RandomForest with SimpleImputer and StandardScaler: 0.8828193626380185
Average of the f1 score RandomForest with SimpleImputer and MinMaxScaler: 0.8661704783009991
Average of the f1 score RandomForest with KNNImputer and StandardS

In [33]:
warnings.filterwarnings("ignore")
steps_xgb = [('imputer', SimpleImputer()),('scaler', StandardScaler()),('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))]
pipeline_xgb = Pipeline(steps_xgb)
param_grid_xgb = {'xgb__learning_rate': [0.01, 0.1],'xgb__max_depth': [3, 4, 5],'xgb__min_child_weight': [1, 2],'xgb__gamma': [0, 0.1, 0.2],
    'xgb__subsample': [0.6, 0.7],'xgb__colsample_bytree': [0.6, 0.7]}
grid_search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=5, scoring='f1_macro')
X_xgb = X[selected_features['XGBoost']]
grid_search_xgb.fit(X_xgb, y)
xbg_best_params = grid_search_xgb.best_params_
print("Best parameters: ", grid_search_xgb.best_params_)
print("Best score: ", grid_search_xgb.best_score_)
steps_rf = [('imputer', SimpleImputer()),('scaler', StandardScaler()),('rf', RandomForestClassifier())]
pipeline_rf = Pipeline(steps_rf)
param_grid_rf = {'rf__n_estimators': [100, 200, 500, 1000],'rf__max_depth': [None, 5, 10, 15]}
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='f1_macro')
grid_search_rf.fit(X, y)
rf_best_params = grid_search_rf.best_params_ 
print("Best parameters: ", grid_search_rf.best_params_)
print("Best score: ", grid_search_rf.best_score_)
steps_dt = [('imputer', SimpleImputer()),('scaler', StandardScaler()),('dt', DecisionTreeClassifier())]
pipeline_dt = Pipeline(steps_dt)
param_grid_dt = {'dt__criterion': ['gini', 'entropy'],'dt__max_depth': [None, 5, 10, 15],'dt__min_samples_split': [2, 5, 10],'dt__min_samples_leaf': [1, 2, 5, 10],
    'dt__max_features': ['auto', 'sqrt', 'log2', None]}
grid_search_dt = GridSearchCV(pipeline_dt, param_grid_dt, cv=5, scoring='f1_macro')
X_dt = X[selected_features['DecisionTree']]
grid_search_dt.fit(X_dt, y)
dt_best_params = grid_search_dt.best_params_
#print("Best parameters: ", grid_search_dt.best_params_)
#print("Best score: ", grid_search_dt.best_score_)

Best parameters:  {'xgb__colsample_bytree': 0.7, 'xgb__gamma': 0, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 5, 'xgb__min_child_weight': 1, 'xgb__subsample': 0.7}
Best score:  0.8639739655822434
Best parameters:  {'rf__max_depth': None, 'rf__n_estimators': 500}
Best score:  0.8729507501945012


In [34]:
# Load the data
data = pd.read_csv("../proj-data-c.csv")
data = data.drop(columns=['referral source:'])
data = data.drop(columns=['[record identification]'])
missing_sex_percentage = (data['sex:'] == '?').mean() * 100
data = data[data['sex:'] != '?']
columns_with_question_mark = data.columns[data.apply(lambda col: (col == '?').any())]
for column in columns_with_question_mark:
    data[column] = data[column].replace('?', np.nan)
data = data.fillna(-1)
data = data[data['age:'] <= 100]
y = data['age:']
data['diagnoses'] = data['diagnoses'].apply(convert_to_target)
data = data.replace('f', 0)
data = data.replace('t', 1)
data = data.replace('M', 0)
data = data.replace('F', 1)
data = data.replace('healthy', 0)
data = data.replace('hyperthyroid condition', 1)
data = data.replace('hypothyroid condition', 2)
data = data.replace('binding protein', 3)
data = data.replace('general health', 4)
data = data.replace('replacement therapy', 5)
data = data.replace('discordant results', 6)
data = data.replace('other', 7)
X = data.drop(columns=['age:'])
X = X.replace('f', 0)
X = X.replace('t', 1)
X = X.replace('M', 0)
X = X.replace('F', 1)
X = X.replace('healthy', 0)
X = X.replace('hyperthyroid condition', 1)
X = X.replace('hypothyroid condition', 2)
X = X.replace('binding protein', 3)
X = X.replace('general health', 4)
X = X.replace('replacement therapy', 5)
X = X.replace('discordant results', 6)
X = X.replace('other', 7)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")
le = LabelEncoder()
y = le.fit_transform(y)
models = {'LinearRegression': LinearRegression(),'Ridge': Ridge(),'Lasso': Lasso(alpha=1.0),'DecisionTreeRegressor': DecisionTreeRegressor(),
    'KNN': KNeighborsRegressor(n_neighbors=10, weights='distance')}
selected_features = {}
for name, model in models.items():
    start_time = time.time()
    sfs = SequentialFeatureSelector(model, n_features_to_select=8, n_jobs=-1)
    sfs.fit(X, y)
    mask = sfs.get_support()
    selected_features[name] = X.columns[mask]
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Features for {name}: {selected_features[name]}')
    print(f'Time taken for {name}: {elapsed_time} seconds\n')

Features for LinearRegression: Index(['sick:', 'pregnant:', 'psych:', 'TSH measured:', 'T3 measured:', 'T3:',
       'TT4:', 'FTI:'],
      dtype='object')
Time taken for LinearRegression: 5.473840951919556 seconds

Features for Ridge: Index(['sick:', 'pregnant:', 'psych:', 'TSH measured:', 'T3 measured:', 'T3:',
       'TT4:', 'FTI:'],
      dtype='object')
Time taken for Ridge: 2.5327494144439697 seconds

Features for Lasso: Index(['sex:', 'TSH:', 'T3:', 'TT4 measured:', 'TT4:', 'FTI:', 'TBG:',
       'diagnoses'],
      dtype='object')
Time taken for Lasso: 2.5410025119781494 seconds

Features for DecisionTreeRegressor: Index(['sick:', 'pregnant:', 'I131 treatment:', 'hypopituitary:', 'psych:',
       'TSH measured:', 'T3 measured:', 'T3:'],
      dtype='object')
Time taken for DecisionTreeRegressor: 2.878739356994629 seconds

Features for KNN: Index(['on antithyroid medication:', 'pregnant:', 'lithium:', 'hypopituitary:',
       'psych:', 'TSH measured:', 'T3:', 'FTI measured:'],
 

In [36]:
import warnings
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")
selected_features = {'LinearRegression': ['query hyperthyroid:', 'lithium:', 'goitre:', 'tumor:','hypopituitary:', 'psych:', 'TSH:', 'T4U:'],
    'Ridge': ['on antithyroid medication:', 'pregnant:', 'thyroid surgery:','I131 treatment:', 'query hyperthyroid:', 'tumor:', 'TSH:','T4U measured:'],
    'Lasso': ['sex:', 'on thyroxine:', 'query on thyroxine:','on antithyroid medication:', 'sick:', 'TSH:', 'TT4:', 'FTI:'],
    'DecisionTreeRegressor': ['sex:', 'on thyroxine:', 'query on thyroxine:','on antithyroid medication:', 'sick:', 'pregnant:', 'TSH measured:','T3:'],
    'KNN': ['sex:', 'on thyroxine:', 'query on thyroxine:','on antithyroid medication:', 'sick:', 'pregnant:', 'T3 measured:','T3:'],}
models = {'LinearRegression': LinearRegression(),'Ridge': Ridge(),'Lasso': Lasso(alpha=1.0),'DecisionTreeRegressor': DecisionTreeRegressor(),
    'KNN': KNeighborsClassifier(n_neighbors=10, weights='distance')}
imputers = [('SimpleImputer', SimpleImputer(strategy='mean')), ('KNNImputer', KNNImputer(n_neighbors=2, weights="uniform"))]
scalers = [('StandardScaler', StandardScaler()), ('MinMaxScaler', MinMaxScaler())]
for model_name, model in models.items():
    best_features = selected_features[model_name]
    X_best = X[best_features]
    X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.2, random_state=42)
    for imputer_name, imputer in imputers:
        X_train_imputed = imputer.fit_transform(X_train)
        X_test_imputed = imputer.transform(X_test)
    for scaler_name, scaler in scalers:
        X_train_scaled = scaler.fit_transform(X_train_imputed)
        X_test_scaled = scaler.transform(X_test_imputed)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        print(f'MSE of {model_name} with {imputer_name} and {scaler_name}: {mean_squared_error(y_test, y_pred)}')

MSE of LinearRegression with KNNImputer and StandardScaler: 348.90916497995005
MSE of LinearRegression with KNNImputer and MinMaxScaler: 348.90916497995005
MSE of Ridge with KNNImputer and StandardScaler: 343.851092549785
MSE of Ridge with KNNImputer and MinMaxScaler: 343.83885754805226
MSE of Lasso with KNNImputer and StandardScaler: 349.87467322801245
MSE of Lasso with KNNImputer and MinMaxScaler: 351.75550866644494
MSE of DecisionTreeRegressor with KNNImputer and StandardScaler: 340.22395472836905
MSE of DecisionTreeRegressor with KNNImputer and MinMaxScaler: 340.6841237424536
MSE of KNN with KNNImputer and StandardScaler: 921.7225352112677
MSE of KNN with KNNImputer and MinMaxScaler: 954.1197183098592


In [37]:
warnings.filterwarnings("ignore")
steps_ridge = [('imputer', KNNImputer()),('scaler', StandardScaler()),('Ridge', Ridge())]
pipeline_ridge = Pipeline(steps_ridge)
param_grid_ridge = {
    'Ridge__alpha': [0.1, 1.0, 10.0],
    'Ridge__fit_intercept': [True, False],
    'Ridge__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}
grid_search_ridge = GridSearchCV(pipeline_ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
X_ridge = X[selected_features['Ridge']]
grid_search_ridge.fit(X_ridge, y)
ridge_best_params = grid_search_ridge.best_params_
print("Best parameters: ", grid_search_ridge.best_params_)
print("Best score: ", grid_search_ridge.best_score_)

Best parameters:  {'Ridge__alpha': 10.0, 'Ridge__fit_intercept': True, 'Ridge__solver': 'sag'}
Best score:  -346.864224452166


## O3

In [38]:
data = pd.read_csv("../proj-data-c.csv")
data = data.drop(columns=['referral source:'])
data = data.drop(columns=['[record identification]'])
missing_sex_percentage = (data['sex:'] == '?').mean() * 100
data = data[data['sex:'] != '?']
columns_with_question_mark = data.columns[data.apply(lambda col: (col == '?').any())]
for column in columns_with_question_mark:
    data[column] = data[column].replace('?', np.nan)
y = data['sex:']
X = data.drop(columns=['sex:'])
X = X.replace('f', 0)
X = X.replace('t', 1)
X = X.fillna(-1)
le = LabelEncoder()
X['diagnoses'] = le.fit_transform(X['diagnoses'])
y = y.replace('M',0)
y = y.replace('F',1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
import warnings
warnings.filterwarnings("ignore")
models = {'LogisticRegression': LogisticRegression(),'XGBoost': XGBClassifier(),'RandomForest': RandomForestClassifier(n_estimators=100),
    'DecisionTree': DecisionTreeClassifier(),'SVM': SVC(),'KNN': KNeighborsClassifier(n_neighbors=10, weights='distance')}
selected_features = {}
for name, model in models.items():
    start_time = time.time()
    sfs = SequentialFeatureSelector(model, n_features_to_select=8, n_jobs=-1)
    sfs.fit(X, y)
    mask = sfs.get_support()
    selected_features[name] = X.columns[mask]
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Features for {name}: {selected_features[name]}')
    #print(f'Time taken for {name}: {elapsed_time} seconds\n')

Features for LogisticRegression: Index(['on thyroxine:', 'query on thyroxine:', 'goitre:', 'tumor:', 'psych:',
       'TSH:', 'T3 measured:', 'TT4:'],
      dtype='object')
Features for XGBoost: Index(['pregnant:', 'thyroid surgery:', 'lithium:', 'hypopituitary:', 'psych:',
       'TSH measured:', 'T4U measured:', 'T4U:'],
      dtype='object')
Features for RandomForest: Index(['on thyroxine:', 'pregnant:', 'thyroid surgery:', 'query hyperthyroid:',
       'lithium:', 'tumor:', 'psych:', 'FTI measured:'],
      dtype='object')
Features for DecisionTree: Index(['on thyroxine:', 'pregnant:', 'query hypothyroid:',
       'query hyperthyroid:', 'lithium:', 'tumor:', 'hypopituitary:',
       'psych:'],
      dtype='object')
Features for SVM: Index(['on thyroxine:', 'pregnant:', 'query hyperthyroid:', 'lithium:',
       'tumor:', 'psych:', 'TSH measured:', 'T4U:'],
      dtype='object')
Features for KNN: Index(['on antithyroid medication:', 'pregnant:', 'query hypothyroid:',
       'tumor:',

In [40]:
selected_features = {'LogisticRegression': ['on thyroxine:', 'query on thyroxine:', 'goitre:', 'tumor:', 'psych:','TSH:', 'T3 measured:', 'TT4:'],
    'XGBoost': ['pregnant:', 'thyroid surgery:', 'lithium:', 'hypopituitary:', 'psych:','TSH measured:', 'T4U measured:', 'T4U:'],
    'RandomForest':['query on thyroxine:', 'pregnant:', 'lithium:', 'tumor:', 'psych:','T4U:', 'FTI measured:', 'TBG measured:'],
    'DecisionTree': ['on thyroxine:', 'pregnant:', 'query hypothyroid:','query hyperthyroid:', 'lithium:', 'tumor:', 'hypopituitary:','psych:'],
    'SVM':['on thyroxine:', 'pregnant:', 'query hyperthyroid:', 'lithium:','tumor:', 'psych:', 'TSH measured:', 'T4U:'],
    'KNN': ['on antithyroid medication:', 'pregnant:', 'query hypothyroid:','tumor:', 'hypopituitary:', 'psych:', 'T3 measured:', 'TBG measured:']}
models = [('LogisticRegression', LogisticRegression()),('XGBoost', XGBClassifier()),('RandomForest', RandomForestClassifier(n_estimators=100)),
    ('DecisionTree', DecisionTreeClassifier()),('SVM', SVC()),
    ('KNN', KNeighborsClassifier(n_neighbors=10, weights='distance')),]
scaling = [('MinMaxScaler', MinMaxScaler()),('StandardScaler', StandardScaler())] 
imput = [
    ('SimpleImputer',SimpleImputer(missing_values=np.nan, strategy='median')), 
    ('KNNImputer',KNNImputer(n_neighbors=2, weights="uniform"))
    ]
# Loop over the models
for model_name, model in models:
    best_features = selected_features[model_name]
    X_best = X[best_features]
    X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.2, random_state=42)
    for scaler_name, scaler in scaling:
        for imput_name, imputer in imput:
            X_train_imputed = imputer.fit_transform(X_train)
            X_test_imputed = imputer.transform(X_test)

            X_train_scaled = scaler.fit_transform(X_train_imputed)
            X_test_scaled = scaler.transform(X_test_imputed)
            
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            print(f'{model_name} with {scaler_name} and {imput_name}:',f1_score(y_test, y_pred, average='macro'))


LogisticRegression with MinMaxScaler and SimpleImputer: 0.4655242212043855
LogisticRegression with MinMaxScaler and KNNImputer: 0.4655242212043855
LogisticRegression with StandardScaler and SimpleImputer: 0.4651466750182691
LogisticRegression with StandardScaler and KNNImputer: 0.4651466750182691
XGBoost with MinMaxScaler and SimpleImputer: 0.5516832936429743
XGBoost with MinMaxScaler and KNNImputer: 0.5516832936429743
XGBoost with StandardScaler and SimpleImputer: 0.5516832936429743
XGBoost with StandardScaler and KNNImputer: 0.5516832936429743
RandomForest with MinMaxScaler and SimpleImputer: 0.5346300159255194
RandomForest with MinMaxScaler and KNNImputer: 0.5365891726428247
RandomForest with StandardScaler and SimpleImputer: 0.5387100459945738
RandomForest with StandardScaler and KNNImputer: 0.5370802378303365
DecisionTree with MinMaxScaler and SimpleImputer: 0.4630002296025175
DecisionTree with MinMaxScaler and KNNImputer: 0.4630002296025175
DecisionTree with StandardScaler and Si

In [41]:
param_grid = {
    'XGB__learning_rate': [0.01, 0.1, 0.3],
    'XGB__max_depth': [3, 6, 9],
    'XGB__min_child_weight': [1, 3, 5],
    'XGB__gamma': [0, 0.1, 0.2],
    'XGB__subsample': [0.5, 0.7, 1.0],
    'XGB__colsample_bytree': [0.5, 0.7, 1.0]
}
imputer = KNNImputer()
scaler = MinMaxScaler()
log_r = XGBClassifier()
grid_search = GridSearchCV(estimator=log_r, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)
grid_search.fit(X_train_scaled, y_train)
print("Best parameters for KNN: ",  grid_search.best_params_)
y_pred = grid_search.predict(X_test_scaled)

Fitting 3 folds for each of 729 candidates, totalling 2187 fits
Best parameters for KNN:  {'XGB__colsample_bytree': 0.5, 'XGB__gamma': 0, 'XGB__learning_rate': 0.01, 'XGB__max_depth': 3, 'XGB__min_child_weight': 1, 'XGB__subsample': 0.5}


## Model Testing

In [42]:
##O1
def convert_to_target(diagnoses):
    if diagnoses == '-':
        return 'healthy'
    elif diagnoses in 'ABCD':
        return 'hyperthyroid condition'
    elif diagnoses in 'EFGH':
        return 'hypothyroid condition'
    elif diagnoses in 'IJ':
        return 'binding protein'
    elif diagnoses in 'K':
        return 'general health'
    elif diagnoses in 'LMN':
        return 'replacement therapy'
    elif diagnoses == 'R':
        return 'discordant results'
    else:
        return 'other'
def apply_O1(data_test, data_results):
    data_test = data_test.drop(columns=['referral source:'])
    data_test = data_test.drop(columns=['[record identification]'])
    drop_indices = data_test[data_test['sex:'] == '?'].index
    data_test = data_test[data_test['sex:'] != '?']
    columns_with_question_mark = data_test.columns[data_test.apply(lambda col: (col == '?').any())]
    for column in columns_with_question_mark:
        data_test[column] = data_test[column].replace('?', np.nan)
    data_results['target'] = data_results['diagnoses'].apply(convert_to_target)
    y = data_results['target']
    y = y.drop(drop_indices)
    X = data_test
    le = LabelEncoder()
    y = le.fit_transform(y)
    X = X.replace('f', 0)
    X = X.replace('t', 1)
    X = X.replace('M', 0)
    X = X.replace('F', 1)
    X = X.fillna(-1)
    best_features_xgb = ['on thyroxine:', 'pregnant:', 'thyroid surgery:', 'TSH:', 'T3:', 'TT4:','FTI:', 'TBG:']  # replace with your actual feature names
    X_selected_xgb = X[best_features_xgb]
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_selected_xgb, y, test_size=0.2, random_state=42)
    best_params_xgb = {'xgb__colsample_bytree': 0.7, 'xgb__gamma': 0.2, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 5, 'xgb__min_child_weight': 1, 'xgb__subsample': 0.7}
    model = Pipeline([('imputer', KNNImputer(weights="distance")),('scaler', StandardScaler()),
    ('classifier', XGBClassifier(learning_rate=best_params_xgb['xgb__learning_rate'],max_depth=best_params_xgb['xgb__max_depth'],
        min_child_weight=best_params_xgb['xgb__min_child_weight'],gamma=best_params_xgb['xgb__gamma'],subsample=best_params_xgb['xgb__subsample'],
        colsample_bytree=best_params_xgb['xgb__colsample_bytree']))])
    model.fit(X_train1, y_train1)   
    y_pred1 = model.predict(X_test1)
    print("Accuracy for Model XGBoost on Diagnoses Problem: \n", f1_score(y_test1, y_pred1, average='macro'))  
apply_O1(pd.read_csv('../proj-test-data.csv'),pd.read_csv('../proj-test-class.csv'))

   

Accuracy for Model XGBoost on Diagnoses Problem: 
 1.0
